In [49]:
# Tensorflow / Keras
from tensorflow import keras # for building Neural Networks
print('Tensorflow/Keras: %s' % keras.__version__) # print version
from keras.models import Sequential # for creating a linear stack of layers for our Neural Network
from keras import Input # for instantiating a keras tensor
from keras.layers import Masking,Bidirectional, LSTM, RepeatVector, Dense, TimeDistributed # for creating layers inside the Neural Network
from keras.optimizers import Adam
# Data manipulation
import pandas as pd # for data manipulation
print('pandas: %s' % pd.__version__) # print version
import numpy as np # for data manipulation
print('numpy: %s' % np.__version__) # print version

# Sklearn
import sklearn
print('sklearn: %s' % sklearn.__version__) # print version
from sklearn.preprocessing import MinMaxScaler # for feature scaling

# Visualization
import plotly 
import plotly.express as px
import plotly.graph_objects as go
print('plotly: %s' % plotly.__version__) # print version

#file accessing
import os
# time stuff
from datetime import timedelta
import calendar


Tensorflow/Keras: 2.9.0
pandas: 1.4.2
numpy: 1.22.4
sklearn: 1.1.1
plotly: 5.9.0


In [50]:
#get displacement for one dot over the whole time 60 20 20
#open file by file
CSVpath="InSAR_data_south/displacement/CSV/test/"
dir_list=[]
lon_south=pd.read_csv('InSAR_data_south/longitude.csv')
lat_south=pd.read_csv('InSAR_data_south/latitude.csv')
for f in os.listdir(CSVpath):
    name, ext=os.path.splitext(f)
    if(ext=='.csv'):
        dir_list.append(f)

dir_list.sort()
print(dir_list)


['20141108.csv', '20141202.csv', '20141226.csv', '20150212.csv', '20150308.csv', '20150401.csv', '20150425.csv', '20150519.csv', '20150612.csv', '20150706.csv', '20150730.csv', '20150823.csv', '20150916.csv']


In [51]:
# df=pd.DataFrame()
# for f in dir_list:
#     name, ext=os.path.splitext(f)
#     if(ext=='.csv'):
#         print(CSVpath+f)
#         df_temp=pd.read_csv(CSVpath+f)
#         df_temp.columns=lon_south.columns
#         df_temp.index=lat_south.columns[:-1]
#         df_temp=df_temp.unstack().reset_index()
#         df_temp.columns=['Longitude','Latitude',name]
#         #print(df1.head())
#         #put it onto the main'
#         df1=pd.DataFrame(data=df_temp.iloc[:,2:3])
#         df1=df_temp.set_index([df_temp.columns[0],df_temp.columns[1]])
        
        
#         df=pd.concat([df,df1], axis=1)
df=pd.read_csv('InSAR_data_south/displacement/export_dataframe1.csv')



In [52]:
# Convert dates to year-months
#df['Year-Month']= (pd.to_datetime(df['Date'], yearfirst=True)).dt.strftime('%Y-%m')
df=df.set_index([df.columns[0],df.columns[1]])
df.columns=pd.to_datetime(df.columns, format='%Y%m%d')
dftest=df
timestep=10
#dftest


In [53]:
#df

In [54]:
#df.query("Longitude==-117.6414 and Latitude==37.6236")
# print(dftest.query('20141108'))
#df   

In [55]:
tup=(-119.9588,35.185)
#tup=(-119.2990,35.8852)
#tup=(-120.2888,34.8349)
#tup=(-117.6414,37.6236)
#print(dftest.loc[tup])

In [56]:
def shaping(datain, timestep):
    
    # Convert input dataframe to array and flatten
    arr=datain.to_numpy().flatten() 
    arr[np.isnan(arr)] = -100
    #print(arr)
    cnt=0
    for mth in range(0, len(datain.columns)-(2*timestep)+1): # Define range lenght of the dates - 2* amount of timesep?? +1
        cnt=cnt+1 # Gives us the number of samples. Later used to reshape the data
        X_start=mth # Start month for inputs of each sample
        X_end=mth+timestep # End month for inputs of each sample
        Y_start=mth+timestep # Start month for targets of each sample. Note, start is inclusive and end is exclusive, that's why X_end and Y_start is the same number
        Y_end=mth+2*timestep # End month for targets of each sample.  
        
        # Assemble input and target arrays containing all samples
        if mth==0:
            X_comb=arr[X_start:X_end]
            Y_comb=arr[Y_start:Y_end]
        else: 
            X_comb=np.append(X_comb, arr[X_start:X_end])
            Y_comb=np.append(Y_comb, arr[Y_start:Y_end])
    
    # Reshape input and target arrays
    X_out=np.reshape(X_comb, (cnt, timestep, 1))
    Y_out=np.reshape(Y_comb, (cnt, timestep, 1))
    return X_out, Y_out

In [57]:
##### Step 2 - Prepare data

# Split data into train and test dataframes
df_train_temp=dftest.iloc[:, :88].copy()
df_test_temp=dftest.iloc[:, 88:110].copy()
#pick a location to test
df_train = df_train_temp[df_train_temp.index==tup].copy()
df_test = df_test_temp[df_test_temp.index==tup].copy()

print(df_test.columns)
#print(df_test)
#df_train.columns=['Date','Disp']
#df_test.columns=['Date','Disp']


# # Split data into train and test dataframes
# df_train=df2_pivot.iloc[:, 0:-2*timestep].copy()
# df_test=df2_pivot.iloc[:, -2*timestep:].copy()

# # Select one location
# dfloc_train = df_train[df_train.index==location].copy()
# dfloc_test = df_test[df_test.index==location].copy()
# #print(df_test.columns)
# Select one location
#dfloc_train = df_train[df_train.index==location].copy()
#dfloc_test = df_test[df_test.index==location].copy()

# Use previously defined shaping function to reshape the data for LSTM

X_train, Y_train = shaping(datain=df_train, timestep= timestep)
X_test, Y_test = shaping(datain=df_test, timestep=timestep)

DatetimeIndex(['2018-05-27', '2018-06-08', '2018-06-20', '2018-07-02',
               '2018-07-14', '2018-07-26', '2018-08-07', '2018-08-19',
               '2018-08-31', '2018-09-12', '2018-09-24', '2018-10-06',
               '2018-10-18', '2018-10-30', '2018-11-11', '2018-11-17',
               '2018-11-23', '2018-12-05', '2018-12-17', '2018-12-29',
               '2019-01-10', '2019-01-22'],
              dtype='datetime64[ns]', freq=None)


In [58]:
#print("dftrain:", df_train)

#print("dftest:", df_test)

#print("Xtrain:", X_train)

#print("Ytrain:", Y_train.shape[2])

#print("Xtrain:", X_train.size)

In [59]:
##### Step 3 - Specify the structure of a Neural Network
model = Sequential(name="LSTM-Model") # Model
model.add(Masking(mask_value=-100, input_shape=(timestep, 1)))
model.add(Input(shape=(X_train.shape[1],X_train.shape[2]), name='Input-Layer')) # Input Layer - need to speicfy the shape of inputs
model.add(Bidirectional(LSTM(units=32, activation='tanh', recurrent_activation='sigmoid', stateful=False), name='Hidden-LSTM-Encoder-Layer')) # Encoder Layer
model.add(RepeatVector(Y_train.shape[1], name='Repeat-Vector-Layer')) # Repeat Vector
model.add(Bidirectional(LSTM(units=32, activation='tanh', recurrent_activation='sigmoid', stateful=False, return_sequences=True), name='Hidden-LSTM-Decoder-Layer')) # Decoder Layer
model.add(TimeDistributed(Dense(units=1, activation='linear'), name='Output-Layer')) # Output Layer, Linear(x) = x



In [60]:
##### Step 4 - Compile the model
#from keras.optimizers import adam_v2
#optimizer = adam_v2.Adam(learning_rate=1)
optimizer=Adam( 0.001)

model.compile(optimizer='adam', # default='rmsprop', an algorithm to be used in backpropagation
              loss='mean_squared_error', # Loss function to be optimized. A string (name of loss function), or a tf.keras.losses.Loss instance.
              metrics=['MeanSquaredError', 'MeanAbsoluteError'], # List of metrics to be evaluated by the model during training and testing. Each of this can be a string (name of a built-in function), function or a tf.keras.metrics.Metric instance. 
              loss_weights=None, # default=None, Optional list or dictionary specifying scalar coefficients (Python floats) to weight the loss contributions of different model outputs.
              weighted_metrics=None, # default=None, List of metrics to be evaluated and weighted by sample_weight or class_weight during training and testing.
              run_eagerly=None, # Defaults to False. If True, this Model's logic will not be wrapped in a tf.function. Recommended to leave this as None unless your Model cannot be run inside a tf.function.
              steps_per_execution=None # Defaults to 1. The number of batches to run during each tf.function call. Running multiple batches inside a single tf.function call can greatly improve performance on TPUs or small models with a large Python overhead.
        
             )

In [61]:


##### Step 5 - Fit the model on the dataset
history = model.fit(X_train, # input data
                    Y_train, # target data
                    batch_size=None, # Number of samples per gradient update. If unspecified, batch_size will default to 32.
                    epochs=3000, # default=1, Number of epochs to train the model. An epoch is an iteration over the entire x and y data provided
                    verbose=2, # default='auto', ('auto', 0, 1, or 2). Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch. 'auto' defaults to 1 for most cases, but 2 when used with ParameterServerStrategy.
                    callbacks=None, # default=None, list of callbacks to apply during training. See tf.keras.callbacks
                    validation_split=0.2, # default=0.0, Fraction of the training data to be used as validation data. The model will set apart this fraction of the training data, will not train on it, and will evaluate the loss and any model metrics on this data at the end of each epoch. 
                    #validation_data=(X_test, y_test), # default=None, Data on which to evaluate the loss and any model metrics at the end of each epoch. 
                    shuffle=True, # default=True, Boolean (whether to shuffle the training data before each epoch) or str (for 'batch').
                    class_weight=None, # default=None, Optional dictionary mapping class indices (integers) to a weight (float) value, used for weighting the loss function (during training only). This can be useful to tell the model to "pay more attention" to samples from an under-represented class.
                    sample_weight=None, # default=None, Optional Numpy array of weights for the training samples, used for weighting the loss function (during training only).
                    initial_epoch=0, # Integer, default=0, Epoch at which to start training (useful for resuming a previous training run).
                    steps_per_epoch=None, # Integer or None, default=None, Total number of steps (batches of samples) before declaring one epoch finished and starting the next epoch. When training with input tensors such as TensorFlow data tensors, the default None is equal to the number of samples in your dataset divided by the batch size, or 1 if that cannot be determined. 
                    validation_steps=None, # Only relevant if validation_data is provided and is a tf.data dataset. Total number of steps (batches of samples) to draw before stopping when performing validation at the end of every epoch.
                    validation_batch_size=None, # Integer or None, default=None, Number of samples per validation batch. If unspecified, will default to batch_size.
                    validation_freq=100, # default=1, Only relevant if validation data is provided. If an integer, specifies how many training epochs to run before a new validation run is performed, e.g. validation_freq=2 runs validation every 2 epochs.
                    max_queue_size=10, # default=10, Used for generator or keras.utils.Sequence input only. Maximum size for the generator queue. If unspecified, max_queue_size will default to 10.
                    workers=1, # default=1, Used for generator or keras.utils.Sequence input only. Maximum number of processes to spin up when using process-based threading. If unspecified, workers will default to 1.
                    use_multiprocessing=True, # default=False, Used for generator or keras.utils.Sequence input only. If True, use process-based threading. If unspecified, use_multiprocessing will default to False. 
                
                   )



Epoch 1/3000
2/2 - 12s - loss: 15.0686 - mean_squared_error: 15.0686 - mean_absolute_error: 3.4757 - 12s/epoch - 6s/step
Epoch 2/3000
2/2 - 0s - loss: 12.4945 - mean_squared_error: 12.4945 - mean_absolute_error: 3.1311 - 41ms/epoch - 20ms/step
Epoch 3/3000
2/2 - 0s - loss: 10.2749 - mean_squared_error: 10.2749 - mean_absolute_error: 2.7927 - 40ms/epoch - 20ms/step
Epoch 4/3000
2/2 - 0s - loss: 8.2967 - mean_squared_error: 8.2967 - mean_absolute_error: 2.4458 - 40ms/epoch - 20ms/step
Epoch 5/3000
2/2 - 0s - loss: 6.7487 - mean_squared_error: 6.7487 - mean_absolute_error: 2.1411 - 41ms/epoch - 20ms/step
Epoch 6/3000
2/2 - 0s - loss: 5.6156 - mean_squared_error: 5.6156 - mean_absolute_error: 1.8707 - 42ms/epoch - 21ms/step
Epoch 7/3000
2/2 - 0s - loss: 4.8991 - mean_squared_error: 4.8991 - mean_absolute_error: 1.6820 - 40ms/epoch - 20ms/step
Epoch 8/3000
2/2 - 0s - loss: 4.5664 - mean_squared_error: 4.5664 - mean_absolute_error: 1.5823 - 42ms/epoch - 21ms/step
Epoch 9/3000
2/2 - 0s - loss

Epoch 69/3000
2/2 - 0s - loss: 4.2739 - mean_squared_error: 4.2739 - mean_absolute_error: 1.5028 - 40ms/epoch - 20ms/step
Epoch 70/3000
2/2 - 0s - loss: 4.2627 - mean_squared_error: 4.2627 - mean_absolute_error: 1.4987 - 38ms/epoch - 19ms/step
Epoch 71/3000
2/2 - 0s - loss: 4.2498 - mean_squared_error: 4.2498 - mean_absolute_error: 1.5006 - 40ms/epoch - 20ms/step
Epoch 72/3000
2/2 - 0s - loss: 4.2434 - mean_squared_error: 4.2434 - mean_absolute_error: 1.4997 - 38ms/epoch - 19ms/step
Epoch 73/3000
2/2 - 0s - loss: 4.2320 - mean_squared_error: 4.2320 - mean_absolute_error: 1.4967 - 39ms/epoch - 19ms/step
Epoch 74/3000
2/2 - 0s - loss: 4.2236 - mean_squared_error: 4.2236 - mean_absolute_error: 1.4945 - 38ms/epoch - 19ms/step
Epoch 75/3000
2/2 - 0s - loss: 4.2189 - mean_squared_error: 4.2189 - mean_absolute_error: 1.4928 - 39ms/epoch - 19ms/step
Epoch 76/3000
2/2 - 0s - loss: 4.2102 - mean_squared_error: 4.2102 - mean_absolute_error: 1.4911 - 38ms/epoch - 19ms/step
Epoch 77/3000
2/2 - 0s -

Epoch 136/3000
2/2 - 0s - loss: 3.9384 - mean_squared_error: 3.9384 - mean_absolute_error: 1.4383 - 42ms/epoch - 21ms/step
Epoch 137/3000
2/2 - 0s - loss: 3.9411 - mean_squared_error: 3.9411 - mean_absolute_error: 1.4470 - 42ms/epoch - 21ms/step
Epoch 138/3000
2/2 - 0s - loss: 3.9499 - mean_squared_error: 3.9499 - mean_absolute_error: 1.4139 - 41ms/epoch - 20ms/step
Epoch 139/3000
2/2 - 0s - loss: 3.9521 - mean_squared_error: 3.9521 - mean_absolute_error: 1.4406 - 40ms/epoch - 20ms/step
Epoch 140/3000
2/2 - 0s - loss: 3.9142 - mean_squared_error: 3.9142 - mean_absolute_error: 1.4419 - 42ms/epoch - 21ms/step
Epoch 141/3000
2/2 - 0s - loss: 3.9492 - mean_squared_error: 3.9492 - mean_absolute_error: 1.4195 - 41ms/epoch - 20ms/step
Epoch 142/3000
2/2 - 0s - loss: 3.9357 - mean_squared_error: 3.9357 - mean_absolute_error: 1.4487 - 40ms/epoch - 20ms/step
Epoch 143/3000
2/2 - 0s - loss: 3.9431 - mean_squared_error: 3.9431 - mean_absolute_error: 1.4298 - 41ms/epoch - 20ms/step
Epoch 144/3000
2

Epoch 202/3000
2/2 - 0s - loss: 3.7609 - mean_squared_error: 3.7609 - mean_absolute_error: 1.4061 - 41ms/epoch - 20ms/step
Epoch 203/3000
2/2 - 0s - loss: 3.7566 - mean_squared_error: 3.7566 - mean_absolute_error: 1.3968 - 40ms/epoch - 20ms/step
Epoch 204/3000
2/2 - 0s - loss: 3.7808 - mean_squared_error: 3.7808 - mean_absolute_error: 1.4102 - 40ms/epoch - 20ms/step
Epoch 205/3000
2/2 - 0s - loss: 3.7687 - mean_squared_error: 3.7687 - mean_absolute_error: 1.4018 - 42ms/epoch - 21ms/step
Epoch 206/3000
2/2 - 0s - loss: 3.7892 - mean_squared_error: 3.7892 - mean_absolute_error: 1.4203 - 59ms/epoch - 29ms/step
Epoch 207/3000
2/2 - 0s - loss: 3.8132 - mean_squared_error: 3.8132 - mean_absolute_error: 1.4198 - 40ms/epoch - 20ms/step
Epoch 208/3000
2/2 - 0s - loss: 3.7860 - mean_squared_error: 3.7860 - mean_absolute_error: 1.4246 - 40ms/epoch - 20ms/step
Epoch 209/3000
2/2 - 0s - loss: 3.7588 - mean_squared_error: 3.7588 - mean_absolute_error: 1.4082 - 42ms/epoch - 21ms/step
Epoch 210/3000
2

Epoch 269/3000
2/2 - 0s - loss: 3.6729 - mean_squared_error: 3.6729 - mean_absolute_error: 1.3963 - 40ms/epoch - 20ms/step
Epoch 270/3000
2/2 - 0s - loss: 3.6639 - mean_squared_error: 3.6639 - mean_absolute_error: 1.3910 - 40ms/epoch - 20ms/step
Epoch 271/3000
2/2 - 0s - loss: 3.6630 - mean_squared_error: 3.6630 - mean_absolute_error: 1.3919 - 41ms/epoch - 20ms/step
Epoch 272/3000
2/2 - 0s - loss: 3.6699 - mean_squared_error: 3.6699 - mean_absolute_error: 1.4003 - 40ms/epoch - 20ms/step
Epoch 273/3000
2/2 - 0s - loss: 3.6732 - mean_squared_error: 3.6732 - mean_absolute_error: 1.3917 - 41ms/epoch - 20ms/step
Epoch 274/3000
2/2 - 0s - loss: 3.6832 - mean_squared_error: 3.6832 - mean_absolute_error: 1.4046 - 41ms/epoch - 20ms/step
Epoch 275/3000
2/2 - 0s - loss: 3.6725 - mean_squared_error: 3.6725 - mean_absolute_error: 1.3914 - 40ms/epoch - 20ms/step
Epoch 276/3000
2/2 - 0s - loss: 3.6788 - mean_squared_error: 3.6788 - mean_absolute_error: 1.3979 - 42ms/epoch - 21ms/step
Epoch 277/3000
2

...

Epoch 2717/3000
2/2 - 0s - loss: 0.1436 - mean_squared_error: 0.1436 - mean_absolute_error: 0.2811 - 42ms/epoch - 21ms/step
Epoch 2718/3000
2/2 - 0s - loss: 0.1060 - mean_squared_error: 0.1060 - mean_absolute_error: 0.2302 - 43ms/epoch - 21ms/step
Epoch 2719/3000
2/2 - 0s - loss: 0.0933 - mean_squared_error: 0.0933 - mean_absolute_error: 0.2109 - 42ms/epoch - 21ms/step
Epoch 2720/3000
2/2 - 0s - loss: 0.0986 - mean_squared_error: 0.0986 - mean_absolute_error: 0.2220 - 43ms/epoch - 21ms/step
Epoch 2721/3000
2/2 - 0s - loss: 0.1067 - mean_squared_error: 0.1067 - mean_absolute_error: 0.2377 - 44ms/epoch - 22ms/step
Epoch 2722/3000
2/2 - 0s - loss: 0.1151 - mean_squared_error: 0.1151 - mean_absolute_error: 0.2394 - 43ms/epoch - 21ms/step
Epoch 2723/3000
2/2 - 0s - loss: 0.1131 - mean_squared_error: 0.1131 - mean_absolute_error: 0.2310 - 42ms/epoch - 21ms/step
Epoch 2724/3000
2/2 - 0s - loss: 0.1004 - mean_squared_error: 0.1004 - mean_absolute_error: 0.2251 - 41ms/epoch - 20ms/step
Epoch 27

2/2 - 0s - loss: 0.0536 - mean_squared_error: 0.0536 - mean_absolute_error: 0.1428 - 42ms/epoch - 21ms/step
Epoch 2784/3000
2/2 - 0s - loss: 0.0528 - mean_squared_error: 0.0528 - mean_absolute_error: 0.1405 - 42ms/epoch - 21ms/step
Epoch 2785/3000
2/2 - 0s - loss: 0.0540 - mean_squared_error: 0.0540 - mean_absolute_error: 0.1408 - 42ms/epoch - 21ms/step
Epoch 2786/3000
2/2 - 0s - loss: 0.0536 - mean_squared_error: 0.0536 - mean_absolute_error: 0.1422 - 42ms/epoch - 21ms/step
Epoch 2787/3000
2/2 - 0s - loss: 0.0503 - mean_squared_error: 0.0503 - mean_absolute_error: 0.1307 - 42ms/epoch - 21ms/step
Epoch 2788/3000
2/2 - 0s - loss: 0.0512 - mean_squared_error: 0.0512 - mean_absolute_error: 0.1361 - 41ms/epoch - 20ms/step
Epoch 2789/3000
2/2 - 0s - loss: 0.0514 - mean_squared_error: 0.0514 - mean_absolute_error: 0.1341 - 43ms/epoch - 21ms/step
Epoch 2790/3000
2/2 - 0s - loss: 0.0526 - mean_squared_error: 0.0526 - mean_absolute_error: 0.1379 - 43ms/epoch - 21ms/step
Epoch 2791/3000
2/2 - 0s

Epoch 2849/3000
2/2 - 0s - loss: 0.0661 - mean_squared_error: 0.0661 - mean_absolute_error: 0.1757 - 41ms/epoch - 20ms/step
Epoch 2850/3000
2/2 - 0s - loss: 0.0711 - mean_squared_error: 0.0711 - mean_absolute_error: 0.1822 - 40ms/epoch - 20ms/step
Epoch 2851/3000
2/2 - 0s - loss: 0.0715 - mean_squared_error: 0.0715 - mean_absolute_error: 0.1736 - 41ms/epoch - 20ms/step
Epoch 2852/3000
2/2 - 0s - loss: 0.0701 - mean_squared_error: 0.0701 - mean_absolute_error: 0.1768 - 43ms/epoch - 21ms/step
Epoch 2853/3000
2/2 - 0s - loss: 0.0758 - mean_squared_error: 0.0758 - mean_absolute_error: 0.1931 - 41ms/epoch - 20ms/step
Epoch 2854/3000
2/2 - 0s - loss: 0.0820 - mean_squared_error: 0.0820 - mean_absolute_error: 0.1943 - 40ms/epoch - 20ms/step
Epoch 2855/3000
2/2 - 0s - loss: 0.0812 - mean_squared_error: 0.0812 - mean_absolute_error: 0.1995 - 41ms/epoch - 20ms/step
Epoch 2856/3000
2/2 - 0s - loss: 0.0721 - mean_squared_error: 0.0721 - mean_absolute_error: 0.1750 - 41ms/epoch - 20ms/step
Epoch 28

Epoch 2915/3000
2/2 - 0s - loss: 0.0383 - mean_squared_error: 0.0383 - mean_absolute_error: 0.1100 - 42ms/epoch - 21ms/step
Epoch 2916/3000
2/2 - 0s - loss: 0.0382 - mean_squared_error: 0.0382 - mean_absolute_error: 0.1088 - 41ms/epoch - 20ms/step
Epoch 2917/3000
2/2 - 0s - loss: 0.0382 - mean_squared_error: 0.0382 - mean_absolute_error: 0.1105 - 40ms/epoch - 20ms/step
Epoch 2918/3000
2/2 - 0s - loss: 0.0376 - mean_squared_error: 0.0376 - mean_absolute_error: 0.1073 - 42ms/epoch - 21ms/step
Epoch 2919/3000
2/2 - 0s - loss: 0.0378 - mean_squared_error: 0.0378 - mean_absolute_error: 0.1089 - 41ms/epoch - 20ms/step
Epoch 2920/3000
2/2 - 0s - loss: 0.0370 - mean_squared_error: 0.0370 - mean_absolute_error: 0.1047 - 41ms/epoch - 20ms/step
Epoch 2921/3000
2/2 - 0s - loss: 0.0375 - mean_squared_error: 0.0375 - mean_absolute_error: 0.1060 - 41ms/epoch - 20ms/step
Epoch 2922/3000
2/2 - 0s - loss: 0.0370 - mean_squared_error: 0.0370 - mean_absolute_error: 0.1060 - 40ms/epoch - 20ms/step
Epoch 29

2/2 - 0s - loss: 0.0338 - mean_squared_error: 0.0338 - mean_absolute_error: 0.1053 - 41ms/epoch - 20ms/step
Epoch 2982/3000
2/2 - 0s - loss: 0.0336 - mean_squared_error: 0.0336 - mean_absolute_error: 0.1055 - 40ms/epoch - 20ms/step
Epoch 2983/3000
2/2 - 0s - loss: 0.0339 - mean_squared_error: 0.0339 - mean_absolute_error: 0.1049 - 41ms/epoch - 20ms/step
Epoch 2984/3000
2/2 - 0s - loss: 0.0337 - mean_squared_error: 0.0337 - mean_absolute_error: 0.1062 - 40ms/epoch - 20ms/step
Epoch 2985/3000
2/2 - 0s - loss: 0.0362 - mean_squared_error: 0.0362 - mean_absolute_error: 0.1139 - 41ms/epoch - 20ms/step
Epoch 2986/3000
2/2 - 0s - loss: 0.0429 - mean_squared_error: 0.0429 - mean_absolute_error: 0.1318 - 42ms/epoch - 21ms/step
Epoch 2987/3000
2/2 - 0s - loss: 0.0481 - mean_squared_error: 0.0481 - mean_absolute_error: 0.1451 - 41ms/epoch - 20ms/step
Epoch 2988/3000
2/2 - 0s - loss: 0.0461 - mean_squared_error: 0.0461 - mean_absolute_error: 0.1411 - 42ms/epoch - 21ms/step
Epoch 2989/3000
2/2 - 0s

In [62]:
##### Step 6 - Use model to make predictions
# Predict results on training data
pred_train = model.predict(X_train)
# Predict esults on test data
pred_test = model.predict(X_test)

1/1 [==============================] - 0s 21ms/step


In [63]:
##### Step 7 - Print Performance Summary
print("")
print('-------------------- Model Summary --------------------')
model.summary() # print model summary
print("")
print('-------------------- Weights and Biases --------------------')
print("Too many parameters to print but you can use the code provided if needed")
print("")
#for layer in model.layers:
#    print(layer.name)
#    for item in layer.get_weights():
#        print("  ", item)
#print("")

# Print the last value in the evaluation metrics contained within history file
print('-------------------- Evaluation on Training Data --------------------')
for item in history.history:
    print("Final", item, ":", history.history[item][-1])
print("")

# Evaluate the model on the test data using "evaluate"
print('-------------------- Evaluation on Test Data --------------------')
results = model.evaluate(X_test, Y_test)
print("")


-------------------- Model Summary --------------------
Model: "LSTM-Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_3 (Masking)         (None, 10, 1)             0         
                                                                 
 Input-Layer (InputLayer)    multiple                  0         
                                                                 
 Hidden-LSTM-Encoder-Layer (  (None, 64)               8704      
 Bidirectional)                                                  
                                                                 
 Repeat-Vector-Layer (Repeat  (None, 10, 64)           0         
 Vector)                                                         
                                                                 
 Hidden-LSTM-Decoder-Layer (  (None, 10, 64)           24832     
 Bidirectional)                                                  

In [64]:
# Plot average monthly temperatures (actual and predicted) for test (out of time) data
fig = go.Figure()

#Trace for actual temperatures
fig.add_trace(go.Scatter(x=np.array(df_test.columns),
                         y=np.array(df_test.values).flatten(),
                         mode='lines',
                         name='Subsidence - Actual (Test)',
                         opacity=0.8,
                         line=dict(color='black', width=1)
                        ))

# Trace for predicted temperatures
fig.add_trace(go.Scatter(x=np.array(df_test.columns[-timestep:]),
                         y=pred_test.flatten(),
                         mode='lines',
                         name='Subsidence - Predicted (Test)',
                         opacity=0.8,
                         line=dict(color='red', width=1)
                        ))

# Change chart background color
fig.update_layout(dict(plot_bgcolor = 'white'))

# Update axes lines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black',
                 title='Month-Year'
                )

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black',
                 title='Subsidence'
                )

# Set figure title
fig.update_layout(title=dict(text="Subsidence", font=dict(color='black')),
                  legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
                 )
fig.show()